# Bethe-Salpeter equation in QTCI representation with patches (############## TODO: QTCI or QTT?)

In [2]:
import QuanticsGrids as QG
using TensorCrossInterpolation
import TCIAlgorithms as TCIA
using HubbardAtoms
using SparseIR
using Quantics
using ITensors

## Bethe-Salpeter equation

Our objective in this work is computing the Bethe-Salpeter equation (BSE), which relates the full vertex $F^r$, the bare susceptibility ${\Chi_0}^r$ and the channel-irreducible vertex $\Gamma^r$
$$
    F^r_{\nu\nu'\omega} = \Gamma^r_{\nu\nu'\omega} + \frac{1}{\beta^2}\sum_{\nu''\nu'''} F^r_{\nu\nu''\omega} {\Chi_0}^r_{\nu''\nu'''\omega} \Gamma^r_{\nu'''\nu'\omega}.
$$
The sum term is equal to the channel-reducible vertex $\Phi^r \equiv F^r_{\nu\nu'\omega} - \Gamma^r_{\nu\nu'\omega}$.
Here, $r$ denotes one of four (density $r=d$, magnetic $r=m$, singlet $r=s$, triplet $r=t$) channels, $\nu$ (and its primed variants) a fermionic Matsubara frequency (of the form $(2n+1)\pi/\beta$ for some integer $n$) and $\omega$ a bosonic Matsubara frequency (of the form $2m\pi/\beta$ for some integer $m$).
The sums are performed over all fermionic frequencies and $\beta = 1/T$ is inverse temperature.

If we chose to straightforwardly represent our quantities by sampling them in a box centered on the origin, the BSE would be easy to implement: For each bosonic frequency, two matrix-matrix multiplications would suffice.
But as this strategy sooner rather than later hits a wall in terms of computing power and memory requirements when approaching lower temperatures, where a larger frequency box is required to yield the same accuracy target in the BSE, people have been looking for alternative solutions for some time. (############## TODO: sources)

## Quantics tensor trains

In the present work, we utilize a quantics tensor train (QTT) representation [@Shinaoka2023].
A function of a single Matsubara frequency $f_\nu$ can be approximately represented by its values on a $2^R$-frequency mesh $\nu \in \{\pi/\beta, 3\pi/\beta, \ldots, (2^{R+1} - 1)\pi/\beta\}$ (in practice, the frequency box is usually centered on the origin) where $R \in \mathbb{N}$ controls the accuracy of the representation.
Each such $\nu$ can be written as
$$
    \nu = \left(\nu_1 2^R + \nu_2 2^{R-1} + \ldots + \nu_R + 1\right)\pi/\beta
$$
where the $\nu_r \in \{0, 1\}$ are binary variables.
Thus $f$ can be viewed as an order-$R$ tensor,
$$
    f_\nu = f_{\nu_1\nu_2\ldots\nu_R}
$$
which then allows us to perform SVD and convert it into a tensor train (also known as multi-particle state or MPS)
$$
    f_{\nu_1\nu_2\ldots\nu_R} \approx \sum_{\alpha_1=1}^{D_1} \cdots \sum_{\alpha_{R-1}=1}^{D_{R-1}} f^{(1)}_{\nu_1,1\alpha_1} f^{(2)}_{\nu_2,\alpha_1\alpha_2} \cdots f^{(R)}_{\nu_R,\alpha_{R-1}1}. 
$$
Here, $f^{(r)}_{\nu_r,\alpha_{r-1}\alpha_{r}}$ is a $2 \times D_{r-1} \times D_r$ tensor and $D_r$ is the bond dimension between two neighboring tensors.
We call $\alpha_r$ the bond (or internal) indices, $\nu_r$ the external indices and $D = \max_r D_r$ the tensor train's bond dimension.
If it is high enough, $D \sim 2^R$, the tensor is represented exactly, but to compress the original function, we can truncate the tensor and throw away unimportant information.
We will not go into the details of how to find the tensor train representation here.

### Multi-particle operators

A very similar concecpt is that of an multi-particle operator (MPO), which consists of order 4 tensors and has two external indices per tensor.
As the name implies, an MPO can be multiplied onto an MPS (or another MPO), being analogous to a matrix if MPSs are viewed as analogous to vectors:
$$
    (A \cdot f)_{\nu_1\ldots\nu_R} = \sum_{\nu_1',\ldots,\nu_R'} A^{(1),\nu_1'}_{\nu_1} \cdot \ldots \cdot A^{(R),\nu_R'}_{\nu_R} f^{(1)}_{\nu_1'} \cdot \ldots \cdot f^{(R)}_{\nu_R'} = \sum_{\nu_1',\ldots,\nu_R'} \sum_{\alpha_{A,1},\ldots,\alpha_{A,R}} \sum_{\alpha_{f,1},\ldots,\alpha_{f,R}} A^{(1),\nu_1'}_{\nu_1,1\alpha_{A,1}} f^{(1)}_{\nu_1',1\alpha_{f,1}} \cdots A^{(R),\nu_R'}_{\nu_R,\alpha_{A,R}1} f^{(R)}_{\nu_R',\alpha_{f,R}1}
$$

### Multiple variables

To form the MPS of a multivariate function $f_{\nu\nu'}$, we have two choices:

- Interleaved representation: $2R$ tensors of dimension $2 \times D_{r-1} \times D_r$.
    $$
        f_{\nu\nu'} \approx \sum_{\alpha_1=1}^{D_1} \cdots \sum_{\alpha_{2R-1}=1}^{D_{2R-1}} f^{(1)}_{\nu_1,1\alpha_1} f^{(2)}_{\nu_1',\alpha_1\alpha_2} f^{(3)}_{\nu_2,\alpha_2\alpha_3} f^{(4)}_{\nu_2',\alpha_3\alpha_4} \cdots f^{(2R-1)}_{\nu_R,\alpha_{2R-2}\alpha_{2R-1}} f^{(2R)}_{\nu_R',\alpha_{2R-1}1}
    $$
- Fused representation: $R$ tensors of dimension $4 \times D_{r-1} \times D_r$.
    $$
        f_{\nu\nu'} \approx \sum_{\alpha_1=1}^{D_1} \cdots \sum_{\alpha_{R-1}=1}^{D_{R-1}} f^{(1)}_{(\nu_1,\nu_1'),1\alpha_1} f^{(2)}_{(\nu_2,\nu_2'),\alpha_1\alpha_2} \cdots f^{(R)}_{(\nu_R,\nu_R'),\alpha_{R-1}1}
    $$

In the following, we will work in the fused representation.

### Multiplication

To compute the BSE, we need to multiply MPSs by each other.
For clarity, we will use $i$, $j$ and $k$ in this section instead of $\nu$, $\nu'$ and $\nu''$.
$$
    h_{ij} = \sum_k f_{ik} g_{kj}
$$
To do that, we introduce an auxilliary MPO $\hat{f}$ such that
$$
    h_{ij} = \sum_{kl} \hat{f}^{kl}_{ij} g_{kl}
$$
which is given by
$$
    \hat{f}^{kl}_{ij} = \sum_{\alpha_1,\ldots,\alpha_{R_1}} \hat{f}^{(1),(k_1, l_1)}_{(i_1, j_1),1\alpha_1} \hat{f}^{(2),(k_2, l_2)}_{(i_2, j_2),\alpha_1\alpha_2} \cdots \hat{f}^{(R),(k_R, l_R)}_{(i_R, j_R),\alpha_{R-1}1}
$$
with
$$
    \hat{f}^{(r),(k_r, l_r)}_{(i_r, j_r),\alpha_{r-1}\alpha_r} = f^{(r)}_{(k_r, i_r),\alpha_{r-1}\alpha_r} \delta_{l_rj_r}.
$$

To demonstrate the discussed techniques, we choose the Hubbard Atom, where exact expressions for all quantities are known [@Thunström2018] and implemented in the package `HubbardAtoms.jl`.

In [3]:
U = 1.6
beta = 2.3
model = HubbardAtom(U, beta)

HubbardAtom(1.6, 2.3, 6.296538261026656, 0.6400000000000001, 2.3176348522138412)

In [13]:
R = 4
N = 2^R
maxbonddim = 40
grid = QG.InherentDiscreteGrid{3}(R, (-N + 1, -N + 1, -N); step=2, unfoldingscheme=:fused)

base = 2
sitesk = [Index(base, "k=$n") for n in 1:R] # ν
sitesk´ = [Index(base, "k´=$n") for n in 1:R] # ν´
sitesq = [Index(base, "q=$n") for n in 1:R] # ω
sites = collect(collect.(zip(sitesk, sitesk´, sitesq)))

4-element Vector{Vector{Index{Int64}}}:
 [(dim=2|id=719|"k=1"), (dim=2|id=485|"k´=1"), (dim=2|id=707|"q=1")]
 [(dim=2|id=744|"k=2"), (dim=2|id=544|"k´=2"), (dim=2|id=776|"q=2")]
 [(dim=2|id=916|"k=3"), (dim=2|id=399|"k´=3"), (dim=2|id=678|"q=3")]
 [(dim=2|id=198|"k=4"), (dim=2|id=694|"k´=4"), (dim=2|id=345|"q=4")]

In [14]:
function makeverts(ch)
    function fq_full(x, y, z)
        return full_vertex(ch, model, (FermionicFreq(x), FermionicFreq(y), BosonicFreq(z)))
    end
    fI_full = QG.quanticsfunction(ComplexF64, grid, fq_full)

    # we absorb 1/β^2 into the chi0 function
    function fq_chi0(x, y, z)
        return 1 / beta^2 *
            chi0(ch, model, (FermionicFreq(x), FermionicFreq(y), BosonicFreq(z)))
    end
    fI_chi0 = QG.quanticsfunction(ComplexF64, grid, fq_chi0)

    function fq_gamma(x, y, z)
        return gamma(ch, model, (FermionicFreq(x), FermionicFreq(y), BosonicFreq(z)))
    end
    fI_gamma = QG.quanticsfunction(ComplexF64, grid, fq_gamma)

    return fq_full, fq_chi0, fq_gamma, fI_full, fI_chi0, fI_gamma
end

makeverts (generic function with 1 method)

In [15]:
function interpolateverts(fI_chi0, fI_full, fI_gamma)
    localdims = dim.(sites)
    sitedims = [dim.(s) for s in sites]
    pordering = TCIA.PatchOrdering(collect(1:R))
    initialpivots = [QG.origcoord_to_quantics(grid, 0)] # approx center of grid

    full_patches = reshape(
        TCIA.adaptiveinterpolate(TCIA.makeprojectable(Float64, fI_full, localdims), pordering; verbosity=0, maxbonddim, initialpivots),
        sitedims,
    )
    chi0_patches = reshape(
        TCIA.adaptiveinterpolate(TCIA.makeprojectable(Float64, fI_chi0, localdims), pordering; verbosity=0, maxbonddim, initialpivots),
        sitedims,
    )
    gamma_patches = reshape(
        TCIA.adaptiveinterpolate(TCIA.makeprojectable(Float64, fI_gamma, localdims), pordering; verbosity=0, maxbonddim, initialpivots),
        sitedims,
    )

    return full_patches, chi0_patches, gamma_patches
end

interpolateverts (generic function with 1 method)

In [16]:
function makevertsdiagonal(full_patches, chi0_patches, gamma_patches)
    siteskk´_vec = [[x, y] for (x, y) in zip(sitesk, sitesk´)]
    sitesq_vec = [[z] for z in sitesq]
    sites_separateq = [x for pair in zip(siteskk´_vec, sitesq_vec) for x in pair]

    full_mps = TCIA.ProjMPSContainer(Float64, full_patches, sites)
    full_kk´_q = Quantics.rearrange_siteinds(full_mps, sites_separateq)
    full_kk´_qq´ = Quantics.makesitediagonal(full_kk´_q, "q")
    full_ptt = TCIA.ProjTTContainer{Float64}(full_kk´_qq´)

    chi0_mps = TCIA.ProjMPSContainer(Float64, chi0_patches, sites)
    chi0_kk´_q = Quantics.rearrange_siteinds(chi0_mps, sites_separateq)
    chi0_kk´_qq´ = Quantics.makesitediagonal(chi0_kk´_q, "q")
    chi0_kk´_q´q´´ = prime(chi0_kk´_qq´)
    chi0_ptt = TCIA.ProjTTContainer{Float64}(chi0_kk´_q´q´´)

    gamma_mps = TCIA.ProjMPSContainer(Float64, gamma_patches, sites)
    gamma_kk´_q = Quantics.rearrange_siteinds(gamma_mps, sites_separateq)
    gamma_kk´_qq´ = Quantics.makesitediagonal(gamma_kk´_q, "q")
    gamma_kk´_q´´q´´´ = prime(gamma_kk´_qq´, 2)
    gamma_ptt = TCIA.ProjTTContainer{Float64}(gamma_kk´_q´´q´´´)

    diagonal_sites = full_kk´_qq´.sites

    return full_ptt, chi0_ptt, gamma_ptt, diagonal_sites
end

makevertsdiagonal (generic function with 1 method)

In [17]:
function calculatebse(full_ptt, chi0_ptt, gamma_ptt, diagonal_sites)
    pordering = TCIA.PatchOrdering(collect(1:(2R)))

    chi0_gamma_ptt = TCIA.adaptivematmul(chi0_ptt, gamma_ptt, pordering; maxbonddim)
    phi_bse_diagonal = TCIA.adaptivematmul(full_ptt, chi0_gamma_ptt, pordering; maxbonddim)
    phi_bse_diagonal_projmps = TCIA.ProjMPSContainer(Float64, phi_bse_diagonal, diagonal_sites)
    phi_bse_projmps_kk´_q = Quantics.extractdiagonal(phi_bse_diagonal_projmps, "q")
    phi_bse_projmps_kk´q = Quantics.rearrange_siteinds(phi_bse_projmps_kk´_q, sites)
    phi_bse = TCIA.ProjTTContainer{Float64}(phi_bse_projmps_kk´q)

    return phi_bse
end

calculatebse (generic function with 1 method)

In [18]:
function comparereference(phi_bse, fq_full, fq_chi0, fq_gamma)
    # normal multiplication for comparison
    box = [
        (x, y, z) for x in range(-N + 1; step=2, length=N),
        y in range(-N + 1; step=2, length=N), z in range(-N; step=2, length=N)
    ]
    chi0_exact = map(splat(fq_chi0), box)
    full_exact = map(splat(fq_full), box)
    gamma_exact = map(splat(fq_gamma), box)
    phi_normalmul = stack(
        gamma_exact[:, :, i] * chi0_exact[:, :, i] * full_exact[:, :, i] for i in 1:N
    )

    phi_adaptivemul = [phi_bse(QG.origcoord_to_quantics(grid, p)) for p in box]

    return norm(phi_normalmul - phi_adaptivemul) / norm(phi_normalmul)
end

comparereference (generic function with 1 method)

In [19]:
ch_d = DensityChannel()
ch_m = MagneticChannel()
ch_s = SingletChannel()
ch_t = TripletChannel()
channels = (ch_d, ch_m, ch_s, ch_t)

(DensityChannel(), MagneticChannel(), SingletChannel(), TripletChannel())

In [20]:
for ch in channels
    fq_full, fq_chi0, fq_gamma, fI_full, fI_chi0, fI_gamma = makeverts(ch)
    full_patches, chi0_patches, gamma_patches = interpolateverts(fI_chi0, fI_full, fI_gamma)
    full_ptt, chi0_ptt, gamma_ptt, diagonal_sites = makevertsdiagonal(full_patches, chi0_patches, gamma_patches)
    phi_bse = calculatebse(full_ptt, chi0_ptt, gamma_ptt, diagonal_sites)
    error = comparereference(phi_bse, fq_full, fq_chi0, fq_gamma)
    println(ch, ":\t", error)
end

DensityChannel():	0.0006574657717592351
MagneticChannel():	0.0037716534948342324
SingletChannel():	0.011923999534902672
TripletChannel():	3.803564254435737e-15
